<a href="https://www.kaggle.com/code/baotranquoc685/s-p500-stocks-analysis?scriptVersionId=196266846" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# S&P 500 Stocks Analysis
## Goal: Analyze the Stock Values of S&P 500 by Statistical Models

In [3]:
# for data manipulation
import pandas as pd
import numpy as np
# for plotting
from matplotlib import pyplot as plt
import plotly.express as px
import plotly.graph_objects as go # note that github wont be able to display these plots because they are interactive
# for some processing
import math
from datetime import datetime, timedelta

## 1. Import files

#### We gather 3 sets of data:
####  - df_companies: the data set of the information of the 500 companies in S&P 500
####  - df_index: the index value of S&P 500 from 2014 until now
####  - df_stocks: the value of a stock for every company in S&P 500 from 2014 until now

In [ ]:
df_companies = pd.read_csv("kaggle/input/sp500_companies.csv")
df_companies

In [ ]:
df_index = pd.read_csv("kaggle/input/sp500_index.csv")
df_index

In [ ]:
df_stocks = pd.read_csv("kaggle/input/sp500_stocks.csv")
df_stocks

## 2. S&P500 Index Value Plot

In [ ]:
def index_value_by_date_plot():
  # note that df_index must contain "Date" and "S&P500" column
  fig = px.line(
    df_index,
    x = "Date",
    y = "S&P500",
    title = "S&P500 Index Value", 
    height = 400
  )
  # get the button for the time period, making the plot interactive
  fig.update_xaxes(
    rangeslider_visible = False,
    rangeselector = dict(
      buttons = list([
        dict(count = 1, label = "1m", step = "month", stepmode = "backward"),
        dict(count = 6, label = "6m", step = "month", stepmode = "backward"),
        dict(count = 1, label = "1y", step = "year", stepmode = "backward"),
        dict(count = 5, label = "5y", step = "year", stepmode = "backward"),
        dict(step = "all")
      ])
    ),
  )

  fig.show()

index_value_by_date_plot()

In [ ]:
def get_year_open_close(df):
  year_open = df.loc[df["Date"].idxmin(), "S&P500"]
  year_close = df.loc[df["Date"].idxmax(), "S&P500"]
  return pd.Series({"Year_open": year_open, "Year_close": year_close})

def index_annual_return_plot():
  year_lis = [2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023]
  df_index_cp = df_index.copy()
  df_index_cp["Date"] = pd.to_datetime(df_index_cp["Date"])
  df_index_cp["Year"] = df_index_cp["Date"].dt.year
  df_index_filtered = df_index_cp[df_index_cp["Year"].isin(year_lis)] # get only the features that is in the year of year_lis
  # now we get the open and close value of each year
  df_index_year = df_index_filtered.groupby("Year").apply(get_year_open_close, include_groups = False).reset_index().sort_values(by = "Year", ascending = True)
  # calculate annual return by (year_close - year_open) / year_open
  df_index_year["Annual_return_rate"] = (df_index_year["Year_close"] - df_index_year["Year_open"]) / df_index_year["Year_open"]
  # plot
  fig = px.bar(
    df_index_year,
    x = "Year",
    y = "Annual_return_rate",
    barmode = "stack",
    labels = {"Year": "Year", "Annual_return_rate": "Annual Return Rate"},
    title = f"Annual Return Rate of S&P 500 from {year_lis[0]} to {year_lis[-1]}",
    color = "Year"
  )
  fig.update_xaxes(tickmode = "linear")
  fig.update_layout(showlegend = False)
  fig.show()
  # calculate annual return rate from the whole period
  total_annual_return_rate = df_index_year["Annual_return_rate"].sum() * 100
  average_annual_return_rate = total_annual_return_rate / len(year_lis)
  print(f"Total Annual Return Rate is {total_annual_return_rate:.2f}%")
  print(f"Annual Average Return Rate is {average_annual_return_rate:.2f}%")
index_annual_return_plot()


#### While the stock is not changing too much in a 1-month period and 6-month period, we can see that it is increasing gradually from 2014 to now.
#### There was a sudden drop in 2020 (hypothetically because of the COVID pandemic).
#### Over the five-year period from September 2013 to September 2023, the stock experienced a total annual return rate of 102.36%, representing an average annual growth rate of 10.24%.

## 3. Market Cap Analysis

## 3.1. Market Cap by Sector Plot

In [ ]:
def marketcap_by_sector_plot():
  # group the features into their sector, then sum up the marketcap, then reset_index, then sort them by marketcap in descending order
  df_sector_marketcap = df_companies.groupby("Sector").agg({"Marketcap": "sum"}).reset_index().sort_values(by = "Marketcap", ascending = False)
  fig = px.bar(
    df_sector_marketcap,
    x = "Sector",
    y = "Marketcap",
    barmode = "stack",
    labels = {"Sector": "Sector", "Marketcap": "Market Cap"},
    title = "Market Cap by Sector",
    height = 400,
    color = "Sector",
  )
  fig.update_layout(showlegend = False)
  fig.show()

marketcap_by_sector_plot()

#### The Technology sector commands the largest market capitalization, followed by Communication Services and Financial Services. Real Estate and Basic Materials exhibit the lowest market capitalization.
#### The reason for this can be the fact that technogolies are the leading industry for the modern world.

## 3.2. Market Cap by Industry Plot

In [ ]:
def marketcap_by_industry_plot(sector, fig):
  # get all the companies within the sector
  df_current_companies = df_companies[df_companies["Sector"] == sector]
  unique_industries = len(df_current_companies["Industry"].unique())
  # group the features (companies) into their Industry, then sum up the marketcap, then reset_index, then sort them according to the marketcap in descending order
  df_industry_marketcap = df_current_companies.groupby("Industry").agg({"Marketcap": "sum"}).reset_index().sort_values(by = "Marketcap", ascending = False)
  fig.add_trace(
    go.Bar(
      x = df_industry_marketcap["Industry"],
      y = df_industry_marketcap["Marketcap"],
      name = sector
    )
  )

def marketcap_by_multiple_industry_plot():
  # get the unique sectors
  unique_sectors = df_companies["Sector"].unique()
  fig = go.Figure()
  # loop over the sectors and plot their industry
  for sector in unique_sectors:
    marketcap_by_industry_plot(sector, fig)
  # update layout
  fig.update_layout(
    title = "Market Cap of the Industries of the 11 Sectors"
  )
  fig.show()
  
marketcap_by_multiple_industry_plot()


#### Each sector has multiple industries, for example Techonology has Semiconductors, Consumer Electronics, etc.
#### Techonology and Communication Services are the leading sectors regarding market capitalization.
#### The Consumer Cyclical and Industrials sectors boast the most diverse range of industries, with 19 each. In contrast, the Energy sector is relatively narrow, encompassing only 5 industries.

## 3.3. Min-Max Market Cap by Industry Plot

In [ ]:
def minmax_marketcap_by_industry_plot(sector, fig):
  def get_minmax_marketcap(df):
    # get the two rows
    marketcap_min = df.loc[df["Marketcap"].idxmin(), :]
    marketcap_max = df.loc[df["Marketcap"].idxmax(), :]
    # concat the rows so that we have a dataframe that has only the marketcap min and marketcap max
    return pd.concat([marketcap_max, marketcap_min], axis = 1).T

  # get all the companies within the current sector
  df_current_companies = df_companies[df_companies["Sector"] == sector]
  df_industry_marketcap = df_current_companies.groupby("Industry").agg({"Marketcap": "sum"}).reset_index()
  minmax_marketcap = get_minmax_marketcap(df_industry_marketcap)
  
  fig.add_trace(
    go.Bar(
      x = minmax_marketcap["Industry"],
      y = minmax_marketcap["Marketcap"],
      name = sector,
    )
  )

def minmax_marketcap_by_multiple_industry_plot():
  fig = go.Figure()

  unique_sectors = df_companies["Sector"].unique()
  for sector in unique_sectors:
    minmax_marketcap_by_industry_plot(sector, fig)

  fig.update_layout(
    title_text = "Min-Max Market Cap by Industry",
    xaxis_title = "Industry",
    yaxis_title = "Market Cap"
  )
  fig.show()

minmax_marketcap_by_multiple_industry_plot()

#### There is a huge discrepency in the market capitalization between the industries (sub-sector) for all of the sectors. So the market capitalization of a sector highly depends on one of its industries.
#### In the Technology sector, Semiconductors's market capitalization is 120 times Internet Retail's market capitalization. And, the disprepency in the Real Estate is minimal, where REIT - Specialty's market capitalization is merely 15.9 times REIT - Hotel & Motel.

## 3.4. Market Cap by Full-time Employees Plot

In [ ]:
def marketcap_by_fulltime_employee_plot():
  fig = px.scatter(
    df_companies,
    x = "Fulltimeemployees",
    y = "Marketcap",
    hover_name = "Shortname",
    labels = {"Fulltimeemployees": "Full-time Employees", "Marketcap": "Market Cap"},
    title = "Market Cap by Full-time Employees",
    height = 400,
    color = "Shortname"
  )
  fig.update_layout(showlegend = False)
  fig.show()

marketcap_by_fulltime_employee_plot()

#### The majority of companies have less than 250 thousand full-time employees and less than $1 trilion in market capitalization.
#### Walmart and Amazon are the two companies with the most full-time employees while Apple and Microsoft are the two with largest market cap.

## 3.5. Market Cap by City Plot

In [ ]:
def marketcap_by_city_plot():
  # get each city total marketcap and sort them in decreasing order
  df_city_marketcap = df_companies.groupby("City").agg({"Marketcap": "sum"}).reset_index().sort_values(by = "Marketcap", ascending = False)
  fig = px.bar(
    df_city_marketcap,
    x = "City",
    y = "Marketcap",
    barmode = "stack",
    labels = {"City": "City", "Marketcap": "Market Cap"},
    title = "Market Cap by City",
    height = 400
  )
  fig.show()

marketcap_by_city_plot()

#### Mountain View, Santa Clara, and Cupertino are the cities with the highest market capitalization (mainly because of the headquarter of the companies such as Microsoft, Apple, Nvidia, Amazon).

## 3.6. Annual Return Rate by Sector Plot

In [ ]:
def annual_return_by_sector(year_lis):
  df_stocks_cp = df_stocks.copy()
  df_stocks_cp["Date"] = pd.to_datetime(df_stocks_cp["Date"]) # convert the date into object format
  df_stocks_cp["Year"] = df_stocks_cp["Date"].dt.year
  df_stocks_filtered = df_stocks_cp[df_stocks_cp["Year"].isin(year_lis)] # get only the features (rows) that is in the time interval of year_lis

  def get_year_open_close(df):
    year_open = df.loc[df["Date"].idxmin(), "Open"]
    year_close = df.loc[df["Date"].idxmax(), "Close"]
    return pd.Series({"Year_open": year_open, "Year_close": year_close})
  
  # we get the open and close value of each stock for the years in year_lis
  # note that after .apply, only the Symbol, Year, Year_open, Year_close column are in df_year_stocks DataFrame, similar to .agg above
  df_year_stocks = df_stocks_filtered.groupby(["Symbol", "Year"]).apply(get_year_open_close, include_groups = False).reset_index().sort_values(by = "Year", ascending = True)
  
  # now we need to add Year_open and Year_close to df_companies to know which sector, industry the companies are in
  k = len(year_lis)
  df_companies_expand = df_companies.loc[df_companies.index.repeat(k)] # repeat each row in df_companies k times
  df_companies_expand["Year"] = year_lis * len(df_companies) # add the Year column, resulting in a dataframe that for each company, there are k rows

  # now we merge the two dataframes together
  df_merge = pd.merge(df_year_stocks, df_companies_expand, on = ["Symbol", "Year"], how = "left")
  # calculate the annual return
  df_merge["Annual_return_rate"] = (df_merge["Year_close"] - df_merge["Year_open"]) / df_merge["Year_open"]

  # loop through the year_lis and plot
  for year in year_lis:
    df_current_year = df_merge.loc[df_merge["Year"] == year]
    df_sector_annual_return = df_current_year.groupby("Sector").agg({"Annual_return_rate": "mean"}).reset_index()

    fig = go.Figure()

    for index, row in df_sector_annual_return.iterrows():
      sector = row["Sector"]
      annual_return_rate = row["Annual_return_rate"]
      fig.add_trace(
        go.Bar(
          x = [sector],
          y = [annual_return_rate],
          name = sector
        )
      )
    fig.update_layout(
      title = f"Annual Return by Sector - {year}",
      xaxis_title = "Sector",
      yaxis_title = "Annual Return Rate",
      barmode = "group"
    )
    fig.show()
  
annual_return_by_sector([2019, 2020, 2021, 2022, 2023,])


#### The market experienced its most robust annual returns in 2019 and 2021, with all sectors having positive returns exceeding 0.1%. While 2020 and 2023 demonstrated a decent performance, 2022 was the challenging year marked by significant annual returns accross most sectors, with the exception of Energy.
#### Between 2019 and 2023, the Energy and Technology sectors exhibited the most robust performance.

## 4. Top Five Market Capitalization Companies Analysis

In [15]:
def top_ten_marketcap_companies(days):
  # unique the companies
  df_unique_companies = df_companies.drop_duplicates(subset = "Shortname")
  # get the top ten unique companies
  df_top_ten_companies = df_unique_companies.sort_values(by = "Marketcap", ascending = False).iloc[0: 10]
  # print out the top ten marketcap companies
  print(df_top_ten_companies[["Symbol", "Shortname"]])
  df_stocks_cp = df_stocks.copy()
  # get the index of these companies
  df_stocks_filtered = df_stocks_cp[df_stocks_cp["Symbol"].isin(df_top_ten_companies["Symbol"])]
  # only use the data within one year from now
  df_stocks_filtered.loc[:, "Date"] = pd.to_datetime(df_stocks_filtered["Date"]) # the .loc here is to edit the originnal df_stocks_cp to avoid ambiguity
  # get the begin date
  end = datetime.today()
  begin = end - timedelta(days = days)
  df_stocks_filtered = df_stocks_filtered[df_stocks_filtered["Date"] >= begin]
  
  return df_stocks_filtered

## 4.1 Adjusted Close Price Plot

In [ ]:
def adj_close_by_date_plot():
  df_stocks_filtered = top_ten_marketcap_companies(365)
  # get the unique companies' symbols
  company_lis = df_stocks_filtered["Symbol"].unique()
  k = len(company_lis)
  number_of_row = math.ceil(k / 2)
  # init plot
  fig, ax = plt.subplots(number_of_row, 2, figsize = (20, 30))
  
  # loop over the companies and plot the adj close by date plots
  for index, company_symbol in enumerate(company_lis, start = 0):
    # get the data correspond to the current company
    df_current_stocks_filtered = df_stocks_filtered[df_stocks_filtered["Symbol"] == company_symbol]
    # plot
    i = index // 2
    j = index % 2
    ax[i, j].plot(df_current_stocks_filtered["Date"], df_current_stocks_filtered["Adj Close"])
    ax[i, j].set_xlabel("Date")
    ax[i, j].set_ylabel("Adjusted Close")
    ax[i, j].set_title(f"Adjusted Closing Price of {company_symbol}")
  
  plt.tight_layout()
  plt.show()

adj_close_by_date_plot()

#### The closing price of Google, Amazon, Apple, Eli Lilly, Meta, Microsoft, Nvidia and Tesla was increasing from September 2023 to July 2024. In July and August of 2024, all of the listed companies experienced a sudden drop in value. Then in September, they recovered but did not reach the peak price before the drop.
#### It is worth to note that all of these companies are in the Technology sector. So the reason for the sudden drop could be the Intel 13th and 14th gen chip crisis.
#### The closing price of Berkshire Hathaway was increasing in the period of Sep 2023 to July 2024 and experienced so sudden drop like the ones in the Technology sector.

## 4.2 Volume Plot

In [ ]:
def volume_by_date_plot():
  df_stocks_filtered = top_ten_marketcap_companies(365)
  # normalize the volume by 1e6
  df_stocks_filtered["Volume"] /= 1e6
  # get the unique companies' symbols
  company_lis = df_stocks_filtered["Symbol"].unique()
  k = len(company_lis)
  number_of_row = math.ceil(k / 2)
  # init plot
  fig, ax = plt.subplots(number_of_row, 2, figsize = (20, 30))

  for index, company_symbol in enumerate(company_lis, start = 0):
    df_current_stocks_filtered = df_stocks_filtered[df_stocks_filtered["Symbol"] == company_symbol]
    # plot
    i = index // 2
    j = index % 2
    ax[i, j].plot(df_current_stocks_filtered["Date"], df_current_stocks_filtered["Volume"])
    ax[i, j].set_xlabel("Date")
    ax[i, j].set_ylabel("Volume")
    ax[i, j].set_title(f"Volume of {company_symbol} (in Million $)")

  plt.tight_layout()
  plt.show()

volume_by_date_plot()

## 4.3 Average Daily Volume by Company Plot

In [ ]:
def average_daily_volume_by_company_plot():
  df_stocks_filtered = top_ten_marketcap_companies(365)
  df_stocks_filtered = df_stocks_filtered.groupby("Symbol").agg({"Volume": "mean"}).reset_index().sort_values(by = "Volume", ascending = False)
  fig = px.bar(
    df_stocks_filtered,
    x = "Symbol",
    y = "Volume",
    barmode = "stack",
    labels = {"Symbol": "Company", "Volume": "Average Daily Volume"},
    title = "Average Daily Volume by Company",
    height = 400,
    color = "Symbol"
  )
  fig.update_layout(showlegend = False)
  fig.show()

average_daily_volume_by_company_plot()

#### Nvidia is having a relatively large daily volume compared to other companies in the same list of the top ten market capitalization companies. This means that over the last year, Nvida was doing very well in sales which is also reflected in the above closing price plot.

## 5. Conclusions
#### - The S&P 500 index has been increasing over the last ten years (from 2014 to now), making it a favorable invesment.
#### - The market capitalization of Technology sector is the largest while the market capitalization of Basic Materials sector is the smallest.
#### - In each sector, the market capitalization is highly affected by one big industry. For example in Communication Services, Internet Content & Information places the majority regarding market capitalization. And there is a huge discrepency between the industries' market cap in each sector.
#### - The majority of companies in the S&P 500 have less than 500 million full-time employees and less than 1 trillion dollars in market capitalization.
#### - Walmart and Amazon are the two that have the most full-time employees while Apple and Microsoft are the two with largest market capitalization.
#### - In the period of 2019 to 2023, the annual return rate of Technology and Energy perform relatively better than the rest of the sectors.
#### - Over last year (2023 - 2024), the adjusted closing price of the ten largest market capitalization companies is increasing gradually. 
#### - Over last year (2023 - 2024), the daily volumes are always above 2.5 million dollars. NVDA is having the largest average daily volume of 400 million while LLY is the lowest with 3.1 million. 